<a href="https://colab.research.google.com/github/kappandrew2/DataPreProcessing/blob/main/MarketResearch_0_0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model Purpose

Utilize historical value and time attributes to predict the next day's gain or loss value

!Dataset Notes: The dataset for this data solution must come from the following web sit and contain a large historical sample of data. For example:

Begin Date = 12/01/2007 (Be mindful that the last 35 periods (in this case, days) will get chopped off of the bottom of the dataset during data preprocessing)

End Date = Today's current value (to be run an hour before market close)

Ticker = SPY

Train Set = all data except last 60 periods (rows)

Prediction Set = all data from -90 periods (days) to current

https://www.wsj.com/market-data/quotes/index/SPX/historical-prices


In [226]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from datetime import date, datetime, timedelta
from pandas._libs.tslibs.timestamps import Timestamp

#Connect to drive and import data set

Using google drive

Importing historical prices for ticker "SPY"

In [227]:
#Create CSV from data export
#https://www.wsj.com/market-data/quotes/index/SPX/historical-prices

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HistoricalPricesSPY.csv')

print(dataset)

Mounted at /content/drive
          Date     Open    High     Low     Close    Volume
0     07/30/21  437.910  440.06  437.77  438.5100  68951203
1     07/29/21  439.815  441.80  439.81  440.6500  47435340
2     07/28/21  439.680  440.30  437.31  438.8300  52472359
3     07/27/21  439.910  439.94  435.99  439.0100  67397133
4     07/26/21  439.310  441.03  439.26  441.0200  43719191
...        ...      ...     ...     ...       ...       ...
1315  05/10/16  206.720  208.50  206.64  208.4500  77472211
1316  05/09/16  205.570  206.40  205.36  205.8892  74374922
1317  05/06/16  204.060  205.77  203.88  205.7200  89315000
1318  05/05/16  205.560  205.98  204.47  204.9700  67619219
1319  05/04/16  204.990  205.85  204.42  205.0100  92243805

[1320 rows x 6 columns]


#Modifiy dataset Content and Headers

Remove contents not required for this exercise

Renaming columns to remove leading white space

Narrowing the dataset can be done via drop or select, both options are available (comment out the one not in use)

In [228]:
dataset.rename({' Close': 'Close'}, axis=1, inplace = True)
dataset = dataset[['Close', 'Date']]
#dataset = dataset.drop([' Open', ' High', ' Low', ' Volume'], axis = 1)

#Dataset information validation

Validate date frame, column contents and data types

In [229]:
dataset['Date'] = pd.to_datetime(dataset['Date'])

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Close   1320 non-null   float64       
 1   Date    1320 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 20.8 KB


#Change indext to date (for troublshooting)

Moving date to the index assists in visually validating processes are working correctly

!Note: This should be "off" except when troublshooting

In [230]:
#dataset['Date_Index'] = dataset['Date']
#dataset.set_index('Date_Index', inplace=True)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Close   1320 non-null   float64       
 1   Date    1320 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 20.8 KB


#Create time attributes

Time attributes will change date from a continous variable into discrete (a numeric categorical value)

In [231]:
dataset['DOW'] = dataset['Date'].dt.dayofweek
dataset['DOY'] = dataset['Date'].dt.dayofyear
dataset['Week'] = dataset['Date'].dt.week
dataset['Month'] = dataset['Date'].dt.month
dataset['Quarter'] = dataset['Date'].dt.quarter

dataset.dtypes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Close             float64
Date       datetime64[ns]
DOW                 int64
DOY                 int64
Week                int64
Month               int64
Quarter             int64
dtype: object

#Create skip-day gain loss values (and dependant variable #1)

!Note: gain-loss-0 will ultimately end up being the dependant variable but also an independant variable (we will create a new column later and shift it down a row)

1) Calculate the first day's gain loss by subtracting day -1 from day 0

2) Calculate the second day's gain loss by subtracting day -2 from day 0

3) Calculate the third day's gain loss by subtracting day -n from day 0

!Note: This should be turned into a loop using i=n where n = the rows to be processed (now many previous rows)




In [232]:
dataset['gain_loss-0'] = dataset['Close'].diff(-1)
dataset['gain_loss-1'] = dataset['Close'].diff(-2)
#dataset['gain_loss-1'] = dataset['gain_loss-1'].shift(periods=-1, fill_value=0) #Removed these to experiment 
#with switching around the dependant variable rather than the independant variable
dataset['gain_loss-2'] = dataset['Close'].diff(-3) 
#dataset['gain_loss-2'] = dataset['gain_loss-2'].shift(periods=-1, fill_value=0)
dataset['gain_loss-3'] = dataset['Close'].diff(-4) 
#dataset['gain_loss-3'] = dataset['gain_loss-3'].shift(periods=-1, fill_value=0)
dataset['gain_loss-4'] = dataset['Close'].diff(-5) 
#dataset['gain_loss-4'] = dataset['gain_loss-4'].shift(periods=-1, fill_value=0)

print(dataset)
#dataset.dtypes

         Close       Date  DOW  ...  gain_loss-2  gain_loss-3  gain_loss-4
0     438.5100 2021-07-30    4  ...      -0.5000        -2.51        -1.43
1     440.6500 2021-07-29    3  ...      -0.3700         0.71         5.19
2     438.8300 2021-07-28    2  ...      -1.1100         3.37         4.28
3     439.0100 2021-07-27    1  ...       3.5500         4.46         7.95
4     441.0200 2021-07-26    0  ...       6.4700         9.96        16.05
...        ...        ...  ...  ...          ...          ...          ...
1315  208.4500 2016-05-10    1  ...       3.4800         3.44          NaN
1316  205.8892 2016-05-09    0  ...       0.8792          NaN          NaN
1317  205.7200 2016-05-06    4  ...          NaN          NaN          NaN
1318  204.9700 2016-05-05    3  ...          NaN          NaN          NaN
1319  205.0100 2016-05-04    2  ...          NaN          NaN          NaN

[1320 rows x 12 columns]


#Create binary version of skip-day gain loss values (and dependant variable #2)

!Note: gain-loss-0b will ultimately end up being the dependant variable but also an independant variable (we will create a new column later and shift it down a row)

This process changes all gain loss continuous variables into a binary-descrete (dichotomous) variables

!Note - This process should be converted into the previous process when that process is converted into a loop

In [233]:
dataset['gain_loss-0b'] = np.where(dataset['gain_loss-0'] > 0, 1, 0)
dataset['gain_loss-1b'] = np.where(dataset['gain_loss-1'] > 0, 1, 0)
dataset['gain_loss-2b'] = np.where(dataset['gain_loss-2'] > 0, 1, 0)
dataset['gain_loss-3b'] = np.where(dataset['gain_loss-3'] > 0, 1, 0)
dataset['gain_loss-4b'] = np.where(dataset['gain_loss-4'] > 0, 1, 0)

dataset.dtypes

Close                  float64
Date            datetime64[ns]
DOW                      int64
DOY                      int64
Week                     int64
Month                    int64
Quarter                  int64
gain_loss-0            float64
gain_loss-1            float64
gain_loss-2            float64
gain_loss-3            float64
gain_loss-4            float64
gain_loss-0b             int64
gain_loss-1b             int64
gain_loss-2b             int64
gain_loss-3b             int64
gain_loss-4b             int64
dtype: object

#Aggregate the binary skip-day gain loss values

This creates a true categorical value from the binary descrete values.

The theory is that, having binary values for each period (sparce matrix) and an aggregate (categorical), the values will work together to increase the value of this data

!Note = This process should be indluded in the loop mentioned in notes from the above process (future modifications to the data pre-processing procedures)

In [234]:
dataset['gain_loss-total_b'] = dataset['gain_loss-0b'] + dataset['gain_loss-1b'] + dataset['gain_loss-2b'] + dataset['gain_loss-3b'] + dataset['gain_loss-4b']

dataset.head(-1)

,Close,Date,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,gain_loss-total_b
0,438.5100,2021-07-30,4,211,30,7,3,-2.1400,-0.3200,-0.5000,-2.51,-1.43,0,0,0,0,0,0
1,440.6500,2021-07-29,3,210,30,7,3,1.8200,1.6400,-0.3700,0.71,5.19,1,1,0,1,1,4
2,438.8300,2021-07-28,2,209,30,7,3,-0.1800,-2.1900,-1.1100,3.37,4.28,0,0,0,1,1,2
3,439.0100,2021-07-27,1,208,30,7,3,-2.0100,-0.9300,3.5500,4.46,7.95,0,0,1,1,1,3
4,441.0200,2021-07-26,0,207,30,7,3,1.0800,5.5600,6.4700,9.96,16.05,1,1,1,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,206.5000,2016-05-11,2,132,19,5,2,-1.9500,0.6108,0.7800,1.53,1.49,0,1,1,1,1,4
1315,208.4500,2016-05-10,1,131,19,5,2,2.5608,2.7300,3.4800,3.44,NaN,1,1,1,1,0,4
1316,205.8892,2016-05-09,0,130,19,5,2,0.1692,0.9192,0.8792,NaN,NaN,1,1,1,0,0,3
1317,205.7200,2016-05-06,4,127,18,5,2,0.7500,0.7100,NaN,NaN,NaN,1,1,0,0,0,2


#Create daily gain loss and denormalize values

1) Calculate the first day's gain loss by subtracting day -1 from day 0

2) Calculate the second day's gain loss by subtracting day -2 from day -1

3) Calculate the third day's gain loss by subtracting day -n from day -n+1

This process creates a new column and removes the top rows in accordance with the desired "lookback" period - shift over 1 and lift by 1, shift over 2 and lift by 2, shift over n and lift by n

!Note: This should be turned into a loop using i=n where n = the rows to be processed (now many previous rows)


In [235]:
dataset['prior_day-0'] = dataset['gain_loss-0']
#dataset['prior_day-1'] = dataset['prior_day-1'].shift(periods=-1, fill_value=0)#Removed this to experiment 
#with switching around the dependant variable rather than the independant variable
dataset['prior_day-1'] = dataset['gain_loss-0']
dataset['prior_day-1'] = dataset['prior_day-1'].shift(periods=-1, fill_value=0)
dataset['prior_day-2'] = dataset['gain_loss-0']
dataset['prior_day-2'] = dataset['prior_day-2'].shift(periods=-2, fill_value=0)
dataset['prior_day-3'] = dataset['gain_loss-0']
dataset['prior_day-3'] = dataset['prior_day-3'].shift(periods=-3, fill_value=0)
dataset['prior_day-4'] = dataset['gain_loss-0']
dataset['prior_day-4'] = dataset['prior_day-4'].shift(periods=-4, fill_value=0)
dataset.head()

,Close,Date,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,gain_loss-total_b,prior_day-0,prior_day-1,prior_day-2,prior_day-3,prior_day-4
0,438.51,2021-07-30,4,211,30,7,3,-2.14,-0.32,-0.50,-2.51,-1.43,0,0,0,0,0,0,-2.14,1.82,-0.18,-2.01,1.08
1,440.65,2021-07-29,3,210,30,7,3,1.82,1.64,-0.37,0.71,5.19,1,1,0,1,1,4,1.82,-0.18,-2.01,1.08,4.48
2,438.83,2021-07-28,2,209,30,7,3,-0.18,-2.19,-1.11,3.37,4.28,0,0,0,1,1,2,-0.18,-2.01,1.08,4.48,0.91
3,439.01,2021-07-27,1,208,30,7,3,-2.01,-0.93,3.55,4.46,7.95,0,0,1,1,1,3,-2.01,1.08,4.48,0.91,3.49
4,441.02,2021-07-26,0,207,30,7,3,1.08,5.56,6.47,9.96,16.05,1,1,1,1,1,5,1.08,4.48,0.91,3.49,6.09


#Create binary version of daily gain loss values

This process changes all gain loss continuous variables into a binary-descrete (dichotomous) variables

!Note - This process should be converted into the previous process when that process is converted into a loop

In [236]:
dataset['prior_day-0b'] = np.where(dataset['prior_day-0'] > 0, 1, 0)
dataset['prior_day-1b'] = np.where(dataset['prior_day-1'] > 0, 1, 0)
dataset['prior_day-2b'] = np.where(dataset['prior_day-2'] > 0, 1, 0)
dataset['prior_day-3b'] = np.where(dataset['prior_day-3'] > 0, 1, 0)
dataset['prior_day-4b'] = np.where(dataset['prior_day-4'] > 0, 1, 0)
dataset.head()

,Close,Date,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,gain_loss-total_b,prior_day-0,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-0b,prior_day-1b,prior_day-2b,prior_day-3b,prior_day-4b
0,438.51,2021-07-30,4,211,30,7,3,-2.14,-0.32,-0.50,-2.51,-1.43,0,0,0,0,0,0,-2.14,1.82,-0.18,-2.01,1.08,0,1,0,0,1
1,440.65,2021-07-29,3,210,30,7,3,1.82,1.64,-0.37,0.71,5.19,1,1,0,1,1,4,1.82,-0.18,-2.01,1.08,4.48,1,0,0,1,1
2,438.83,2021-07-28,2,209,30,7,3,-0.18,-2.19,-1.11,3.37,4.28,0,0,0,1,1,2,-0.18,-2.01,1.08,4.48,0.91,0,0,1,1,1
3,439.01,2021-07-27,1,208,30,7,3,-2.01,-0.93,3.55,4.46,7.95,0,0,1,1,1,3,-2.01,1.08,4.48,0.91,3.49,0,1,1,1,1
4,441.02,2021-07-26,0,207,30,7,3,1.08,5.56,6.47,9.96,16.05,1,1,1,1,1,5,1.08,4.48,0.91,3.49,6.09,1,1,1,1,1


#Aggregate the binary daily gain loss values

This creates a true categorical value from the binary descrete values.

The theory is that, having binary values for each period (sparce matrix) and an aggregate (categorical), the values will work together to increase the value of this data

!Note = This process should be indluded in the loop mentioned in notes from the above process (future modifications to the data pre-processing procedures)

In [237]:
dataset['prior_day-total_b'] = dataset['prior_day-0b'] + dataset['prior_day-1b'] + dataset['prior_day-2b'] + dataset['prior_day-3b'] + dataset['prior_day-4b'] 
dataset.head(5)

,Close,Date,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,gain_loss-total_b,prior_day-0,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-0b,prior_day-1b,prior_day-2b,prior_day-3b,prior_day-4b,prior_day-total_b
0,438.51,2021-07-30,4,211,30,7,3,-2.14,-0.32,-0.50,-2.51,-1.43,0,0,0,0,0,0,-2.14,1.82,-0.18,-2.01,1.08,0,1,0,0,1,2
1,440.65,2021-07-29,3,210,30,7,3,1.82,1.64,-0.37,0.71,5.19,1,1,0,1,1,4,1.82,-0.18,-2.01,1.08,4.48,1,0,0,1,1,3
2,438.83,2021-07-28,2,209,30,7,3,-0.18,-2.19,-1.11,3.37,4.28,0,0,0,1,1,2,-0.18,-2.01,1.08,4.48,0.91,0,0,1,1,1,3
3,439.01,2021-07-27,1,208,30,7,3,-2.01,-0.93,3.55,4.46,7.95,0,0,1,1,1,3,-2.01,1.08,4.48,0.91,3.49,0,1,1,1,1,4
4,441.02,2021-07-26,0,207,30,7,3,1.08,5.56,6.47,9.96,16.05,1,1,1,1,1,5,1.08,4.48,0.91,3.49,6.09,1,1,1,1,1,5


#Creating Rolling mean attribute values

Rolling mean values are based on daily gain loss and represent the trending direction of the prior n mean values (5, 10, 15, n, row mean values)

the rolling mean works from the top row down - for exampple the mean of row 1 and 2 would appear on row 2. We need the mean of row 1 and 2 to land on row 1. This requires us to reverse the index of each desired mean column. The process to do this creates pandas value lists

!Note: this process can convert into a loop  where n = list of n mean values (as described in the description above)

In [238]:
#Rolling averages based on prior day gain loss
rolling_prior_day = dataset['prior_day-0']

rolling_prior_day_5 = rolling_prior_day[::-1].rolling(5).mean()[::-1]
rolling_prior_day_10 = rolling_prior_day[::-1].rolling(10).mean()[::-1]
rolling_prior_day_15 = rolling_prior_day[::-1].rolling(15).mean()[::-1]
rolling_prior_day_20 = rolling_prior_day[::-1].rolling(20).mean()[::-1]
rolling_prior_day_25 = rolling_prior_day[::-1].rolling(25).mean()[::-1]
rolling_prior_day_30 = rolling_prior_day[::-1].rolling(30).mean()[::-1]

print(rolling_prior_day_10)

0       0.717
1       0.590
2       0.259
3       0.342
4       0.394
        ...  
1315      NaN
1316      NaN
1317      NaN
1318      NaN
1319      NaN
Name: prior_day-0, Length: 1320, dtype: float64


#Create close variance variable and binary

Close variance uses a rolling averages of n days (Close attribute) minus the row's Close value. This indicates when the market is falling below a rolling average. 

The binary is calculated based on 1 = >0 and 0<0

In [239]:
rolling_close = dataset['Close']
#rolling_close_x = (rolling_close[::-1].rolling(x).mean()[::-1]) #left this in for reference
rolling_close_3 = rolling_close - (rolling_close[::-1].rolling(3).mean()[::-1])
rolling_close_6 = rolling_close - (rolling_close[::-1].rolling(6).mean()[::-1])
rolling_close_9 = rolling_close - (rolling_close[::-1].rolling(9).mean()[::-1])
rolling_close_12 = rolling_close - (rolling_close[::-1].rolling(12).mean()[::-1])
rolling_close_15 = rolling_close - (rolling_close[::-1].rolling(15).mean()[::-1])
rolling_close_18 = rolling_close - (rolling_close[::-1].rolling(18).mean()[::-1])
rolling_close_21 = rolling_close - (rolling_close[::-1].rolling(21).mean()[::-1])

rolling_close_3b = pd.DataFrame({'rolling_close_3b': np.where(rolling_close_3 > 0, 1, 0)})
rolling_close_6b = pd.DataFrame({'rolling_close_6b': np.where(rolling_close_6 > 0, 1, 0)})
rolling_close_9b = pd.DataFrame({'rolling_close_9b': np.where(rolling_close_9 > 0, 1, 0)})
rolling_close_12b = pd.DataFrame({'rolling_close_12b': np.where(rolling_close_12 > 0, 1, 0)})
rolling_close_15b = pd.DataFrame({'rolling_close_15b': np.where(rolling_close_15 > 0, 1, 0)})
rolling_close_18b = pd.DataFrame({'rolling_close_18b': np.where(rolling_close_18 > 0, 1, 0)})
rolling_close_21b = pd.DataFrame({'rolling_close_21b': np.where(rolling_close_21 > 0, 1, 0)})

rolling_close_3 = pd.DataFrame({'rolling_close_3': rolling_close_3})
rolling_close_6 = pd.DataFrame({'rolling_close_6': rolling_close_6})
rolling_close_9 = pd.DataFrame({'rolling_close_9': rolling_close_9})
rolling_close_12 = pd.DataFrame({'rolling_close_12': rolling_close_12})
rolling_close_15 = pd.DataFrame({'rolling_close_15': rolling_close_15})
rolling_close_18 = pd.DataFrame({'rolling_close_18': rolling_close_18})
rolling_close_21 = pd.DataFrame({'rolling_close_21': rolling_close_21})

print(rolling_close_3)

      rolling_close_3
0           -0.820000
1            1.153333
2           -0.790000
3           -0.980000
4            2.213333
...               ...
1315         1.763600
1316         0.362800
1317         0.486667
1318              NaN
1319              NaN

[1320 rows x 1 columns]


#Add Rolling Mean attributes to dataset

This is a normal concat funtion

In [240]:
dataset = pd.concat([dataset,
                       rolling_close_3, 
                       rolling_close_6, 
                       rolling_close_9, 
                       rolling_close_12, 
                       rolling_close_15, 
                       rolling_close_18, 
                       rolling_close_21, 
                       rolling_close_3b, 
                       rolling_close_6b, 
                       rolling_close_9b,
                       rolling_close_12b, 
                       rolling_close_15b, 
                       rolling_close_18b, 
                       rolling_close_21b
                       ], axis = 1)

print(dataset_x)

         Close       Date  ...  rolling_close_18b  rolling_close_21b
0     438.5100 2021-07-30  ...                  1                  1
1     440.6500 2021-07-29  ...                  1                  1
2     438.8300 2021-07-28  ...                  1                  1
3     439.0100 2021-07-27  ...                  1                  1
4     441.0200 2021-07-26  ...                  1                  1
...        ...        ...  ...                ...                ...
1315  208.4500 2016-05-10  ...                  0                  0
1316  205.8892 2016-05-09  ...                  0                  0
1317  205.7200 2016-05-06  ...                  0                  0
1318  204.9700 2016-05-05  ...                  0                  0
1319  205.0100 2016-05-04  ...                  0                  0

[1320 rows x 43 columns]


#Create aggregate of rolling close binary

This is simply a sum of all rolling close binary values

In [241]:
dataset['rolling_close_total_b'] =  (dataset['rolling_close_3b'] + 
                                     dataset['rolling_close_6b'] + 
                                     dataset['rolling_close_9b'] + 
                                     dataset['rolling_close_12b'] + 
                                     dataset['rolling_close_15b'] + 
                                     dataset['rolling_close_18b'] + 
                                     dataset['rolling_close_21b']
                                     )

dataset.head(1)



,Close,Date,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,gain_loss-total_b,prior_day-0,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-0b,prior_day-1b,prior_day-2b,prior_day-3b,prior_day-4b,prior_day-total_b,rolling_close_3,rolling_close_6,rolling_close_9,rolling_close_12,rolling_close_15,rolling_close_18,rolling_close_21,rolling_close_3b,rolling_close_6b,rolling_close_9b,rolling_close_12b,rolling_close_15b,rolling_close_18b,rolling_close_21b,rolling_close_total_b
0,438.51,2021-07-30,4,211,30,7,3,-2.14,-0.32,-0.5,-2.51,-1.43,0,0,0,0,0,0,-2.14,1.82,-0.18,-2.01,1.08,0,1,0,0,1,2,-0.82,-1.15,0.84,2.669167,2.576667,2.96,3.415714,0,0,1,1,1,1,1,5


#Remove NaN rows

Need to remove the NaN rows from bottom of dataset. These will cause errors in the analysis if not removed.

Due to this delete, the dataset must contain 30 additional periods (rows) of history beyond what is desired. This was mentioned in the notation heading of this solution. (due to rolling means and shifts).

In [242]:
#dataset.dropna(inplace = True)

dataset.head(-5)

,Close,Date,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,gain_loss-total_b,prior_day-0,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-0b,prior_day-1b,prior_day-2b,prior_day-3b,prior_day-4b,prior_day-total_b,rolling_close_3,rolling_close_6,rolling_close_9,rolling_close_12,rolling_close_15,rolling_close_18,rolling_close_21,rolling_close_3b,rolling_close_6b,rolling_close_9b,rolling_close_12b,rolling_close_15b,rolling_close_18b,rolling_close_21b,rolling_close_total_b
0,438.5100,2021-07-30,4,211,30,7,3,-2.1400,-0.3200,-0.5000,-2.5100,-1.4300,0,0,0,0,0,0,-2.1400,1.8200,-0.1800,-2.0100,1.0800,0,1,0,0,1,2,-0.820000,-1.150000,0.840000,2.669167,2.576667,2.960000,3.415714,0,0,1,1,1,1,1,5
1,440.6500,2021-07-29,3,210,30,7,3,1.8200,1.6400,-0.3700,0.7100,5.1900,1,1,0,1,1,4,1.8200,-0.1800,-2.0100,1.0800,4.4800,1,0,0,1,1,3,1.153333,1.498333,4.484444,4.998333,4.916000,5.410000,6.053333,1,1,1,1,1,1,1,7
2,438.8300,2021-07-28,2,209,30,7,3,-0.1800,-2.1900,-1.1100,3.3700,4.2800,0,0,0,1,1,2,-0.1800,-2.0100,1.0800,4.4800,0.9100,0,0,1,1,1,3,-0.790000,0.695000,3.698889,3.600000,3.744667,3.975000,4.850000,0,1,1,1,1,1,1,6
3,439.0100,2021-07-27,1,208,30,7,3,-2.0100,-0.9300,3.5500,4.4600,7.9500,0,0,1,1,1,3,-2.0100,1.0800,4.4800,0.9100,3.4900,0,1,1,1,1,4,-0.980000,2.170000,4.332222,3.925833,4.216000,4.621667,5.570952,0,1,1,1,1,1,1,6
4,441.0200,2021-07-26,0,207,30,7,3,1.0800,5.5600,6.4700,9.9600,16.0500,1,1,1,1,1,5,1.0800,4.4800,0.9100,3.4900,6.0900,1,1,1,1,1,5,2.213333,6.520000,6.650000,6.226667,6.631333,7.240000,8.171429,1,1,1,1,1,1,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,204.8508,2016-05-17,1,138,20,5,2,-1.9292,0.0908,-1.7092,-1.6492,-3.5992,0,1,0,0,0,1,-1.9292,2.0200,-1.8000,0.0600,-1.9500,0,1,0,1,0,2,-0.612800,-1.466000,-1.202533,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
1311,206.7800,2016-05-16,0,137,20,5,2,2.0200,0.2200,0.2800,-1.6700,0.8908,1,1,1,0,1,4,2.0200,-1.8000,0.0600,-1.9500,2.5608,1,0,1,0,1,3,0.746667,0.290133,0.708978,NaN,NaN,NaN,NaN,1,1,1,0,0,0,0,3
1312,204.7600,2016-05-13,4,134,19,5,2,-1.8000,-1.7400,-3.6900,-1.1292,-0.9600,0,0,0,0,0,0,-1.8000,0.0600,-1.9500,2.5608,0.1692,0,1,0,1,1,3,-1.180000,-1.553200,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
1313,206.5600,2016-05-12,3,133,19,5,2,0.0600,-1.8900,0.6708,0.8400,1.5900,1,0,1,1,1,4,0.0600,-1.9500,2.5608,0.1692,0.7500,1,0,1,1,1,4,-0.610000,0.211800,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,1


#Create dependant variables (2 dependants)

As mentioned earlier, the gain_loss-0 attribute is a dependant variable. It's binary conterpart, gain_loss-0b is also a dependant variable.

the dependant variables need shifted down one row. This will adjust all of the independant variable into a position where they are trying to predict the "day ahead". Because the data is shifted down one day the last day must be removed.

!Note: Due to the organizaiton of this dataset (train and test set being time-based) this adjustment for the dependant variables will create results for next day. 


In [243]:
y = pd.DataFrame(dataset['gain_loss-0']).reset_index(drop = True)
y.loc[-1] = [0]
y.index = y.index + 1
y = y.sort_index()
y.drop(y.tail(1).index, inplace = True)
y.rename(columns={'gain_loss-0': 'y'}, inplace=True)
y_df = pd.DataFrame(y, columns=['y'])

yb = pd.DataFrame(dataset['gain_loss-0b']).reset_index(drop = True)
yb.loc[-1] = [0]
yb.index = yb.index + 1
yb = yb.sort_index()
yb.drop(yb.tail(1).index, inplace = True)
yb.rename(columns={'gain_loss-0b': 'yb'}, inplace=True)
yb_df = pd.DataFrame(yb, columns=['yb'])

print(y)
print("----------------")
print(yb)

           y
0     0.0000
1    -2.1400
2     1.8200
3    -0.1800
4    -2.0100
...      ...
1315 -1.9500
1316  2.5608
1317  0.1692
1318  0.7500
1319 -0.0400

[1320 rows x 1 columns]
----------------
      yb
0      0
1      0
2      1
3      0
4      0
...   ..
1315   0
1316   1
1317   1
1318   1
1319   0

[1320 rows x 1 columns]


In [244]:
a = len(y.index)
b = len(yb.index)
c = len(dataset.index)
d = len(rolling_prior_day_5.index)
a1 = len(y_df.index)
b1 = len(yb_df.index)

print(a)
print(b)
print(c)
print(d)
print(a1)
print(b1)

1320
1320
1320
1320
1320
1320


#Create final dataset and review

A concat procedure is necessary to create the final dataset.

There should be a total of 37 columns

In [245]:
dataset_final = pd.concat([dataset,
           rolling_prior_day_5, 
           rolling_prior_day_10, 
           rolling_prior_day_15, 
           rolling_prior_day_20, 
           rolling_prior_day_25, 
           rolling_prior_day_30,
           y_df,
           yb_df],
           axis = 1)

dataset_final.head(1)

,Close,Date,DOW,DOY,Week,Month,Quarter,gain_loss-0,gain_loss-1,gain_loss-2,gain_loss-3,gain_loss-4,gain_loss-0b,gain_loss-1b,gain_loss-2b,gain_loss-3b,gain_loss-4b,gain_loss-total_b,prior_day-0,prior_day-1,prior_day-2,prior_day-3,prior_day-4,prior_day-0b,prior_day-1b,prior_day-2b,prior_day-3b,prior_day-4b,prior_day-total_b,rolling_close_3,rolling_close_6,rolling_close_9,rolling_close_12,rolling_close_15,rolling_close_18,rolling_close_21,rolling_close_3b,rolling_close_6b,rolling_close_9b,rolling_close_12b,rolling_close_15b,rolling_close_18b,rolling_close_21b,rolling_close_total_b,prior_day-0,prior_day-0,prior_day-0,prior_day-0,prior_day-0,prior_day-0,y,yb
0,438.51,2021-07-30,4,211,30,7,3,-2.14,-0.32,-0.5,-2.51,-1.43,0,0,0,0,0,0,-2.14,1.82,-0.18,-2.01,1.08,0,1,0,0,1,2,-0.82,-1.15,0.84,2.669167,2.576667,2.96,3.415714,0,0,1,1,1,1,1,5,-0.286,0.717,0.199333,0.404,0.5364,0.551333,0.0,0


#Evaluate dataset for NaN

Throught the processes above there should have been some NaN values created at the tail

In [246]:
dataset_final.dropna(inplace = True)

a = len(dataset.index)
e = len(dataset_final.index)

print("rows dropped = {}".format(a-e))

rows dropped = 30


#Create dataset splitting variable (Train and Pred)

Date variables based on today date are required to prevent "hardcoding" dates into the model

The date_var variable will represent the most current date in the dataset. This allows the solution to be run for any timeframe.

The following code can replace the current date_var logic in the case the current method causes issues. Note, this method requires adjustment when back testing 

date_var = pd.to_datetime(date.today()) 

use train/pred_minus_days to tune the model for longer or shorter periods of time. Allow for a 30 day overlap where pred will have 30 days

In [247]:
train_minus_days = 90
pred_minus_days = 120

date_var = dataset_final['Date'].max()
train_begin_date = dataset_final['Date'].min()
train_end_date = (date_var - pd.to_timedelta(train_minus_days, unit='d'))
pred_begin_date = (date_var - pd.to_timedelta(pred_minus_days, unit='d'))
pred_end_date = date_var

train_begin_date = train_begin_date.to_pydatetime()
train_end_date = train_end_date.to_pydatetime()
pred_begin_date = pred_begin_date.to_pydatetime()
pred_end_date = pred_end_date.to_pydatetime()

#train_begin_date = pd.DataFrame([train_begin_date], columns=['train_begin_date'])
#train_end_date = pd.DataFrame([train_end_date], columns=['train_end_date'])
#pred_begin_date = pd.DataFrame([pred_begin_date], columns=['pred_begin_date'])
#pred_end_date = pd.DataFrame([pred_end_date], columns=['pred_end_date'])

print(train_begin_date)
print(train_end_date)
print(pred_begin_date)
print(pred_end_date)

2016-06-16 00:00:00
2021-05-01 00:00:00
2021-04-01 00:00:00
2021-07-30 00:00:00


#Split between training and predict data sets

The top last 90 periods (rows) will generate the pred data set.

All but the top 60 periods (rows) will generate the training data set.

The 30 day overlap can provide a measure of the model's degredation over time

!Note - The market is closed on weekends and holidays. The count of days in each set will NOT equal the amount of days between begin and end dates.

In [248]:
#split text and train datasets
predset = dataset_final[(dataset_final['Date'] >= pred_begin_date) & 
                        (dataset_final['Date'] <= pred_end_date)]
trainset = dataset_final[(dataset_final['Date'] >= train_begin_date) & 
                         (dataset_final['Date'] <= train_end_date)]
type(predset)

pandas.core.frame.DataFrame

In [249]:
list(dataset.columns)

['Close',
 'Date',
 'DOW',
 'DOY',
 'Week',
 'Month',
 'Quarter',
 'gain_loss-0',
 'gain_loss-1',
 'gain_loss-2',
 'gain_loss-3',
 'gain_loss-4',
 'gain_loss-0b',
 'gain_loss-1b',
 'gain_loss-2b',
 'gain_loss-3b',
 'gain_loss-4b',
 'gain_loss-total_b',
 'prior_day-0',
 'prior_day-1',
 'prior_day-2',
 'prior_day-3',
 'prior_day-4',
 'prior_day-0b',
 'prior_day-1b',
 'prior_day-2b',
 'prior_day-3b',
 'prior_day-4b',
 'prior_day-total_b',
 'rolling_close_3',
 'rolling_close_6',
 'rolling_close_9',
 'rolling_close_12',
 'rolling_close_15',
 'rolling_close_18',
 'rolling_close_21',
 'rolling_close_3b',
 'rolling_close_6b',
 'rolling_close_9b',
 'rolling_close_12b',
 'rolling_close_15b',
 'rolling_close_18b',
 'rolling_close_21b',
 'rolling_close_total_b']

#Convert dataset into X and y and refine column membership

This process separates the dependant and independant variables

X should not contain the y or yb attributes

for X, "Date" should be removed since it is a time-series value; date attributes will represent time

for X, "Close" should be removed due to its relationship to the indepenant variable

Two models will come out of this model, one for continuous variable y and binary value yb

In [250]:
X = trainset
X.drop(X.tail(31).index, inplace = True)
X = X.drop(['y','yb', 'Date', 'Close'], axis = 1).values
y = trainset['y'].values
yb = trainset['yb'].values
print(X)
print("-------------------------")
print(y)
print("-------------------------")
print(yb)

[[  4.         120.          17.         ...   0.8345       1.104
    0.86066667]
 [  3.         119.          17.         ...   1.1865       1.3016
    0.76      ]
 [  2.         118.          17.         ...   1.1335       1.116
    0.71633333]
 ...
 [  2.         216.          31.         ...   0.326        0.519212
    0.258     ]
 [  1.         215.          31.         ...   0.35696      0.637968
    0.25664   ]
 [  0.         214.          31.         ...   0.35096      0.5524
    0.34733333]]
-------------------------
[ 0.9    -2.76    2.66   ...  0.2297  0.6308 -1.3908]
-------------------------
[1 0 1 ... 1 1 0]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [251]:
#Validation of row counts

f = len(X)
g = len(y)
h = len(yb)

print(f, g, h)

1196 1196 1196


#Train the models

The model can be extended to use any regression or classificaiton model.

Current model inventory:

1) Random Forest

In [252]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 120, random_state = 0)

regressor.fit(X, y)

regressor_b = RandomForestRegressor(n_estimators = 120, random_state = 0)

regressor_b.fit(X, yb)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=120, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

#Create predict dataset

The predict dataset should match the process used to generate the training dataset

In [253]:
#Prepare predict set
Xpred = predset
Xpred = Xpred[[ 'DOW', 'Month', 'Quarter', 'gain_loss-0', 'gain_loss-1', 
               'gain_loss-2', 'gain_loss-3', 'gain_loss-4', 'gain_loss-total_b', 
               'prior_day-0', 'prior_day-1', 'prior_day-2', 'prior_day-3', 'prior_day-4', 
               'prior_day-total_b', 'rolling_close_3', 'rolling_close_6', 'rolling_close_9', 
               'rolling_close_12', 'rolling_close_15', 'rolling_close_18', 'rolling_close_21', 
               'rolling_close_total_b'
               ]].values

Xpred_b = Xpred[[ 'DOW', 'Month', 'Quarter', 
                 'gain_loss-0b', 'gain_loss-1b', 'gain_loss-2b', 'gain_loss-3b', 
                 'gain_loss-4b', 'prior_day-0b', 'prior_day-1b', 'prior_day-2b', 
                 'prior_day-3b', 'prior_day-4b', 'rolling_close_3b', 'rolling_close_6b', 
                 'rolling_close_9b', 'rolling_close_12b', 'rolling_close_15b', 'rolling_close_18b', 
                 'rolling_close_21b'
                 ]].values

y_actual = predset['y'].values
yb_actual = predset['yb'].values
print(Xpred)
print("-------------------------")
print(y_actual)
print("-------------------------")
print(yb_actual)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  app.launch_new_instance()


IndexError: ignored

In [ ]:
i = len(Xpred)
j = len(y_actual)
k = len(yb_actual)

print(i, j, k)

#Generate predictions

Predictions are made for both continuous and binary

In [ ]:
y_pred = regressor.predict(Xpred)

yb_pred = regressor_b.predict(Xpred)

np_array = np.concatenate((y_pred.reshape(len(y_pred),1), 
                           yb_pred.reshape(len(y_pred),1),
                           y_actual.reshape(len(y_actual),1),
                           yb_actual.reshape(len(yb_actual),1),
                           ), axis = 1)

results = pd.DataFrame(np_array, columns = ['y_pred', 'yb_pred', 'y_actual', 'yb_actual'])

print(results)


#Create buy/sell indicator/strategy based on pred

1 = Buy next day

0 = Sell next day

yb_pred_num is a number that can help tune the model. Any number above .5 is less resk adverse (creates more risk)

In [ ]:
yb_pred_num = 0.50

results['y_pred_arg'] = np.where(results['y_pred'] > 0, 1, 0)
results['yb_pred_arg'] = np.where(results['yb_pred'] > yb_pred_num, 1, 0)
results['y_actual_arg'] = np.where(results['y_actual'] > 0, 1, 0)
results['yb_acutal_arg'] = np.where(results['yb_actual'] > 0, 1, 0)

print(results)


#Add in 'short' ticker for same period

A short ticker is one that behavies opposite of the selected ticker in this evaluation.

The concept of having a short ticker is to trade "into" it when trading "out" of the primary ticker

Note! - This section is not broke out; use hashtag notes as a reference

In [ ]:
#import dataset
dataset_short = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HistoricalPricesHIBS.csv')
#rename values with leading white spaces
dataset_short.rename({' Close': 'Close'}, axis=1, inplace = True)
#create gain loss value for short
dataset_short['gain_loss_short-0'] = dataset_short['Close'].diff(-1)
#select out values needed for model
dataset_short = dataset_short[['Close', 'Date', 'gain_loss_short-0']]
#convert date to datetype
dataset_short['Date'] = pd.to_datetime(dataset_short['Date'])
#select out mating dates to pred dataset
pred_short = dataset_short[(dataset_short['Date'] >= pred_begin_date) & 
                         (dataset_short['Date'] <= pred_end_date)]
#move rows down by one
y_short = pd.DataFrame(pred_short['gain_loss_short-0']).reset_index(drop = True)
y_short.loc[-1] = [0]
y_short.index = y_short.index + 1
y_short = y_short.sort_index()
y_short.drop(y_short.tail(1).index, inplace = True)
y_short.rename(columns={'gain_loss_short-0': 'y_short'}, inplace=True)
y_short_df = pd.DataFrame(y_short, columns=['y_short'])
#add short to results
results = pd.concat([results,
                    y_short_df],
                    axis = 1)
#verfity y_short and y_actual are both 0 at row
print(results)

In [ ]:
type(results)

#Model Performance

To account accurately the top row of the "results" dataset must be removed

Model performanced is based on the buy-sell relationship between the "..._arg" columns

Each model is measured as well as a various combination of the models.

In [ ]:
results['y_buy'] = np.where(results['y_pred_arg'] >= 1, results['y_actual'], results['y_short'])
results['yb_buy'] = np.where(results['yb_pred_arg'] >= 1, results['y_actual'], results['y_short'])
results['yoryb_buy'] = np.where((results['y_pred_arg'] + results['yb_pred_arg']) >= 1, results['y_actual'], results['y_short'])
results['yandyb_buy'] = np.where((results['y_pred_arg'] + results['yb_pred_arg']) >= 2, results['y_actual'], results['y_short'])


The following section should be turned into a loop

In [ ]:
performance_all = results
performance_all = performance_all[1:] #Remove top row since we do not have actual value
performance_all = performance_all.sum(axis = 0)
print(performance_all)

In [ ]:
performance_30 = results.head(31)
performance_30 = performance_30[1:] #Remove top row since we do not have actual value
performance_30 = performance_30.sum(axis = 0)
print(performance_30)

In [ ]:
performance_60 = results.head(61)
performance_60 = performance_60[1:] #Remove top row since we do not have actual value
performance_60 = performance_60.sum(axis = 0)
print(performance_60)

In [ ]:
performance_15 = results.head(16)
performance_15 = performance_15[1:] #Remove top row since we do not have actual value
performance_15 = performance_15.sum(axis = 0)
print(performance_15)

In [ ]:
performance_45 = results.head(46)
performance_45 = performance_45[1:] #Remove top row since we do not have actual value
performance_45 = performance_45.sum(axis = 0)
print(performance_45)

In [ ]:
performance_5 = results.head(6)
performance_5 = performance_5[1:] #Remove top row since we do not have actual value
performance_5 = performance_5.sum(axis = 0)
print(performance_5)

In [ ]:
performance_10 = results.head(11)
performance_10 = performance_10[1:] #Remove top row since we do not have actual value
performance_10 = performance_10.sum(axis = 0)
print(performance_10)

In [ ]:
performance_20 = results.head(21)
performance_20 = performance_20[1:] #Remove top row since we do not have actual value
performance_20 = performance_20.sum(axis = 0)
print(performance_20)

In [ ]:
performance_25 = results.head(26)
performance_25 = performance_25[1:] #Remove top row since we do not have actual value
performance_25 = performance_25.sum(axis = 0)
print(performance_25)

In [ ]:
performance_35 = results.head(36)
performance_35 = performance_35[1:] #Remove top row since we do not have actual value
performance_35 = performance_35.sum(axis = 0)
print(performance_35)

In [ ]:
performance_40 = results.head(41)
performance_40 = performance_40[1:] #Remove top row since we do not have actual value
performance_40 = performance_40.sum(axis = 0)
print(performance_40)

In [ ]:
performance_50 = results.head(51)
performance_50 = performance_50[1:] #Remove top row since we do not have actual value
performance_50 = performance_50.sum(axis = 0)
print(performance_50)

In [ ]:
performance_55 = results.head(56)
performance_55 = performance_55[1:] #Remove top row since we do not have actual value
performance_55 = performance_55.sum(axis = 0)
print(performance_55)

In [ ]:
performance = pd.concat([performance_5,
                         performance_10,
                         performance_15, 
                         performance_20,
                         performance_25,
                         performance_30,
                         performance_35,
                         performance_40, 
                         performance_45,
                         performance_50,
                         performance_55,
                         performance_60, 
                         performance_all], axis=1)

print(performance)

In [ ]:
df1 = performance.T
print(df1)

In [ ]:
plt.plot(df1['y_actual'], color='black')
plt.plot(df1['yoryb_buy'], color='blue')
plt.plot(df1['yb_buy'], color='red')

In [ ]:
#results.to_csv('/content/drive/MyDrive/Colab Notebooks/results_sky_hibs_20121201-20210431_.csv')
#performance.to_csv('/content/drive/MyDrive/Colab Notebooks/performance_sky_hibs_20121201-20210431_.csv')